# Cortex XDR API Scrapping
This notebook implements routines to explore the Palo Alto's Cortex XDR API.

## Requirements
Requires a valid API key for Cortex XDR.  Use the Google Colab's Secret feature to store this key and name it following the pattern:

`<key-type>_<key_id>`

Where `key_type` should be `standard` or `advanced` and the `<key_id>` should be an integer, just like informed by Cortex when the key is created.  You'll need to inform these values right on the first step so the notebook can build the key label to retrieve it from Secrets.

In [ ]:
#@title Setup

from sys import argv
from datetime import datetime, timezone
from secrets import choice
from string import ascii_letters, digits
from hashlib import sha256
from json import dumps
from http.client import HTTPSConnection

from google.colab import userdata


KEY_TYPE = 'advanced'  #@param['standard', 'advanced']
KEY_ID = 37            #@param{type:'integer'}
FQDN = 'api-acme.xdr.us.paloaltonetworks.com'  #@param{type:'string'}

key_label = f'{KEY_TYPE}_{KEY_ID}'

try:
  key = userdata.get(key_label)
except:  # don't wanna import SecretNotFoundError
  print(f'🔴 Key `{key_label}` not found')
else:
  print(f'🟢 Libraries and API key `{key_label}` loaded')

In [ ]:
#@title Functions

def get_headers(key_id, key, key_type):
  headers = {}
  if key_type == 'advanced':
    nonce = ''.join([choice(ascii_letters + digits) for _ in range(64)])
    timestamp = int(datetime.now(timezone.utc).timestamp()) * 1000
    auth_key = '%s%s%s' % (key, nonce, timestamp)
    headers = {
      'x-xdr-timestamp': str(timestamp),
      'x-xdr-nonce': nonce,
      'x-xdr-auth-id': str(key_id),
      'Authorization': sha256(auth_key.encode('utf-8')).hexdigest()
    }
  else:
    headers = {
      'Authorization': key,
      'x-xdr-auth-id': str(key_id)
    }
    return headers

def get_payload(start, end):
  # Lots of room for improvement here
  payload = {
    'request_data': {
    'filters': [
      {
      'field': 'severity',
      'operator': 'in',
      'value': [
        'low',
        'medium',
        'high'
    ]
      }
    ],
    'search_from': start,
    'search_to': end,
    'sort': {
      'field': 'creation_time',
      'keyword': 'desc'
    }
    }
  }
  return payload

def request(key_id, key, key_type, fqdn, endpoint, start, end):
  payload = get_payload(start, end)
  headers = get_headers(key_id, key, key_type)
  conn = HTTPSConnection(fqdn)
  conn.request('POST', endpoint, dumps(payload), headers)
  res = conn.getresponse()
  data = res.read()
  return data.decode('utf-8')

def paginator(key_id, key, key_type, fqdn, endpoint, start, count, pages):
  page = 1
  base_name = f'cortex-alerts-{datetime.now().isoformat()}'
  while page <= pages:
    fname = f'{base_name}-{page}.json'
    end = start + count
    print(f'[{page:02d}/{pages:02d}] ', end='')
    with open(fname, 'w') as f:
      try:
        res = request(key_id, key, key_type, fqdn, endpoint, start, end)
      except:
        print('Error fetching data from Cortex')
        break
      f.write(res)
      print(f'Page {page} ({start:03d}-{end:03d}) stored in {fname}')
    page += 1
    start += count


print(f'🟢 Loaded all basic functions')

In [ ]:
#@title Get Alerts
#@markdown Reference: https://cortex-panw.stoplight.io/docs/cortex-xdr/813e387002342-get-all-alerts

ENDPOINT = '/public_api/v1/alerts/get_alerts_multi_events'  #@param{type:'string'}
START    = 0    #@param{type:'integer'}
COUNT    = 100  #@param{type:'integer'}
PAGES    = 10   #@param{type:'integer'}


paginator(
  KEY_ID,
  key,
  KEY_TYPE,
  FQDN,
  ENDPOINT,
  START,
  COUNT,
  PAGES
)